# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [32]:
# импортируем все необходимые библиотеки
import pandas as pd
import numpy as np
import nltk
import time
import re
from tqdm import notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier

import warnings
warnings.filterwarnings('ignore')

RND = 1 #для фиксации рандомайзера(random_state=RND)

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0) # index_col - установили индекс датафрейма
display(df.head())
df.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [3]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 81.1 MB


In [4]:
# посмотрим на кол-во дубликатов
print(df.duplicated().sum())
# посмотрим на кол-во пропусков
df.isna().sum()

0


text     0
toxic    0
dtype: int64

Дубликатов и пропусков в данных нету

Посмотрим на соотношение позитивных и негативных комментариев

In [5]:
display(df['toxic'].value_counts())
df['toxic'].value_counts(normalize=True) # содержит значения относительно частоты встречаемых значений

0    143106
1     16186
Name: toxic, dtype: int64

0    0.898388
1    0.101612
Name: toxic, dtype: float64

**Предобработка данных**

Напишем функцию для лемматизации, удаления всех лишних символов кроме латинских букв и для удаления лишних пробелов

In [6]:
notebook.tqdm.pandas() 
def text_clear(text):
    clear_text = re.sub(r'[^a-zA-Z]', ' ', text)
    clear_text = ' '.join(clear_text.split())
    return clear_text

df['text'] = df['text'].apply(text_clear)#очищенный текст
df['text'].head() 

0    Explanation Why the edits made under my userna...
1    D aww He matches this background colour I m se...
2    Hey man I m really not trying to edit war It s...
3    More I can t make any real suggestions on impr...
4    You sir are my hero Any chance you remember wh...
Name: text, dtype: object

In [7]:
%%time
notebook.tqdm.pandas() 
# загружаем пакет omw-1.4 библиотеки nltk 
# для корректной работы функции lemmatizer.lemmatize()
nltk.download('omw-1.4')
# загрузим инструмент маркировки из библиотеки nltk, 
# который потребуется для удаления прилагательных
nltk.download('averaged_perceptron_tagger')

# объявляем класс WordNetLemmatizer()
lemmatizer = WordNetLemmatizer()

# создадим функцию для лемматизации датасета
def lemmatize_text(text):
    tokens_tagged = nltk.pos_tag(nltk.word_tokenize(text))#токенизируем текст и возвращаем кортеж [токен, nltk_tag]
    lemmatized_text = []
    for word, tag in tokens_tagged:
        if tag.startswith('J'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'a'))#лемматизация прилагательных
        elif tag.startswith('V'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'v'))#лемматизация глаголов
        elif tag.startswith('N'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'n'))#лемматизация существительных
        elif tag.startswith('R'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'r'))#лемматизация наречий
        else:
            lemmatized_text.append(lemmatizer.lemmatize(word))#если тэги не найдены, выполнить неспецифическую лемматизацию
    return " ".join(lemmatized_text)#возвращаем нетокенизированный текст

# лемматизируем текст сообщений обучающего признака
df['text'] = df['text'].apply(lambda x: lemmatize_text(x))

[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


CPU times: user 12min 55s, sys: 5.79 s, total: 13min 1s
Wall time: 14min 52s


Разделим датафрейм на две выборки: обучающую и тестовую в соотношение 75%:25%

In [8]:
features = df['text']
target = df['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size=0.25, random_state=RND)

features_train.shape, features_test.shape, target_train.shape, target_test.shape

((119469,), (39823,), (119469,), (39823,))

Удалим stopwords в выборках: features_train и features_test

In [9]:
stopwords = set(stopwords.words('english'))

Получим мешки слов двумя способами, чтобы протестировать модели на разных вариантах данных и выяснить какой способ преобразования данных покажет наилучший результат

Мешок слов для признаков методом TfidfVectorizer

In [10]:
%time
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train_tfv = count_tf_idf.fit_transform(features_train)
features_test_tfv = count_tf_idf.transform(features_test)
features_train_tfv.shape, features_test_tfv.shape

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.78 µs


((119469, 135333), (39823, 135333))

Мешок слов для признаков методом CountVectorizer

In [11]:
%time
count_vec = CountVectorizer(stop_words=stopwords)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs


In [12]:
features_train_cv = count_vec.fit_transform(features_train)
features_test_cv = count_vec.transform(features_test)
features_train_cv.shape, features_test_cv.shape

((119469, 135333), (39823, 135333))

In [13]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 65.8 MB


**Вывод:**
* данные импортированы с удалением лишнего индекса при чтении
* дубликатов, пропусков нету
* в данных виден большой дисбаланс классов
* проведена лемматизация данных и удаление лишних символов
* данные поделены на две выборки: обучающую и тестовую
* удалены стоп-слова из признаков
* получены мешки слов в признаках двумя способами

## Обучение

In [14]:
cv = KFold(5, shuffle=True, random_state=RND)

### Обучение моделей на корпусе с TfidVectorizer

**Модель LogisticRegression**

In [15]:
%%time

params_lin_tfv = {"class_weight": ['balanced'], 
             "max_iter": [50, 100, 150, 200]}

model_lin_tfv = LogisticRegression(random_state=RND)

model_lin_rscv_tfv = RandomizedSearchCV(
    model_lin_tfv, params_lin_tfv, cv=cv, scoring='f1', 
    random_state=RND, refit = True, n_jobs=-1)
model_lin_rscv_tfv.fit(features_train_tfv, target_train)

best_model_lin_tfv = model_lin_rscv_tfv.best_estimator_
best_params_model_lin_tfv = model_lin_rscv_tfv.best_params_
best_score_lin_tfv = abs(model_lin_rscv_tfv.best_score_)

print(f'F1 на train = {best_score_lin_tfv}')

F1 на train = 0.7431893022567043
CPU times: user 6min 14s, sys: 7min 32s, total: 13min 46s
Wall time: 17min 12s


**Модель DecisionTreeClassifier**

In [16]:
%%time

params_tree_tfv = {
          "max_depth": [10, 30, 50],   
          "class_weight": ['balanced']
         }
model_tree_tfv = DecisionTreeClassifier(random_state=RND)

model_tree_rscv_tfv = RandomizedSearchCV(model_tree_tfv, params_tree_tfv, cv=cv, scoring='f1',
                                       random_state=RND, refit = True, n_jobs=-1)
model_tree_rscv_tfv.fit(features_train_tfv, target_train)

best_model_tree_tfv = model_tree_rscv_tfv.best_estimator_
best_params_model_tree_tfv = model_tree_rscv_tfv.best_params_
best_score_tree_tfv = abs(model_tree_rscv_tfv.best_score_)

print(f'F1 на train = {best_score_tree_tfv}')

F1 на train = 0.6231634466875364
CPU times: user 9min 41s, sys: 0 ns, total: 9min 41s
Wall time: 9min 57s


**Модель RandomForestClassifier**

In [17]:
%%time

params_forest_tfv = {
          "max_depth": [5, 10, 15], 
          "n_estimators": [30, 60, 90],  
          "class_weight": ['balanced']
         }
model_forest_tfv = RandomForestClassifier(random_state=RND)

model_forest_rscv_tfv = RandomizedSearchCV(model_forest_tfv, params_forest_tfv, cv=cv, scoring='f1',
                                       random_state=RND, refit = True, n_jobs=-1)
model_forest_rscv_tfv.fit(features_train_tfv, target_train)

best_model_forest_tfv = model_forest_rscv_tfv.best_estimator_
best_params_model_forest_tfv = model_forest_rscv_tfv.best_params_
best_score_forest_tfv = abs(model_forest_rscv_tfv.best_score_)

print(f'F1 на train = {best_score_forest_tfv}')

F1 на train = 0.3678696428988617
CPU times: user 11min 21s, sys: 0 ns, total: 11min 21s
Wall time: 11min 30s


**Модель CatBoostClassifier**

In [18]:
%%time

params_cat_tfv = {
          "max_depth": [1, 2, 3], 
          "n_estimators": [10, 20, 30],  
          "auto_class_weights": ['Balanced']
         }
model_cat_tfv = CatBoostClassifier(logging_level = 'Silent', random_state=RND)

model_cat_rscv_tfv = RandomizedSearchCV(model_cat_tfv, params_cat_tfv, cv=cv, scoring='f1',
                                       random_state=RND, refit = True, n_jobs=-1)
model_cat_rscv_tfv.fit(features_train_tfv, target_train)

best_model_cat_tfv = model_cat_rscv_tfv.best_estimator_
best_params_model_cat_tfv = model_cat_rscv_tfv.best_params_
best_score_cat_tfv = abs(model_cat_rscv_tfv.best_score_)

print(f'F1 на train = {best_score_cat_tfv}')

F1 на train = 0.6828816112319964
CPU times: user 33min 32s, sys: 1.74 s, total: 33min 33s
Wall time: 34min 19s


### Обучение моделей на корпусе с CountVectorizer

**Модель LogisticRegression**

In [19]:
%%time

params_lin_cv = {"class_weight": ['balanced'], 
             "max_iter": [50, 100, 150, 200]}

model_lin_cv = LogisticRegression(random_state=RND)

model_lin_rscv_cv = RandomizedSearchCV(
    model_lin_cv, params_lin_cv, cv=cv, scoring='f1', 
    random_state=RND, refit = True, n_jobs=-1)
model_lin_rscv_cv.fit(features_train_cv, target_train)

best_model_lin_cv = model_lin_rscv_cv.best_estimator_
best_params_model_lin_cv = model_lin_rscv_cv.best_params_
best_score_lin_cv = abs(model_lin_rscv_cv.best_score_)

print(f'F1 на train = {best_score_lin_cv}')

F1 на train = 0.7527597428380723
CPU times: user 9min 24s, sys: 11min 49s, total: 21min 13s
Wall time: 21min 15s


In [20]:
print(best_model_lin_cv)
best_params_model_lin_cv

LogisticRegression(class_weight='balanced', max_iter=200, random_state=1)


{'max_iter': 200, 'class_weight': 'balanced'}

**Модель DecisionTreeClassifier**

In [21]:
%%time

params_tree_cv = {
          "max_depth": [10, 30, 50],   
          "class_weight": ['balanced']
         }
model_tree_cv = DecisionTreeClassifier(random_state=RND)

model_tree_rscv_cv = RandomizedSearchCV(model_tree_cv, params_tree_cv, cv=cv, scoring='f1',
                                       random_state=RND, refit = True, n_jobs=-1)
model_tree_rscv_cv.fit(features_train_cv, target_train)

best_model_tree_cv = model_tree_rscv_cv.best_estimator_
best_params_model_tree_cv = model_tree_rscv_cv.best_params_
best_score_tree_cv = abs(model_tree_rscv_cv.best_score_)

print(f'F1 на train = {best_score_tree_cv}')

F1 на train = 0.6559146056372608
CPU times: user 6min 7s, sys: 0 ns, total: 6min 7s
Wall time: 6min 8s


**Модель RandomForestClassifier**

In [22]:
%%time

params_forest_cv = {
          "max_depth": [5, 10, 15], 
          "n_estimators": [30, 60, 90],  
          "class_weight": ['balanced']
         }
model_forest_cv = RandomForestClassifier(random_state=RND)

model_forest_rscv_cv = RandomizedSearchCV(model_forest_cv, params_forest_cv, cv=cv, scoring='f1',
                                       random_state=RND, refit = True, n_jobs=-1)
model_forest_rscv_cv.fit(features_train_cv, target_train)

best_model_forest_cv = model_forest_rscv_cv.best_estimator_
best_params_model_forest_cv = model_forest_rscv_cv.best_params_
best_score_forest_cv = abs(model_forest_rscv_cv.best_score_)

print(f'F1 на train = {best_score_forest_cv}')

F1 на train = 0.36396434570334846
CPU times: user 8min 24s, sys: 49.4 ms, total: 8min 24s
Wall time: 8min 25s


**Модель CatBoostClassifier**

In [23]:
%%time

params_cat_cv = {
          "max_depth": [1, 2, 3], 
          "n_estimators": [10, 20, 30],  
          "auto_class_weights": ['Balanced']
         }
model_cat_cv = CatBoostClassifier(logging_level = 'Silent', random_state=RND)

model_cat_rscv_cv = RandomizedSearchCV(model_cat_cv, params_cat_cv, cv=cv, scoring='f1',
                                       random_state=RND, refit = True, n_jobs=-1)
model_cat_rscv_cv.fit(features_train_cv, target_train)

best_model_cat_cv = model_cat_rscv_cv.best_estimator_
best_params_model_cat_cv = model_cat_rscv_cv.best_params_
best_score_cat_cv = abs(model_cat_rscv_cv.best_score_)

print(f'F1 на train = {best_score_cat_cv}')

F1 на train = 0.6855539429272997
CPU times: user 27min 11s, sys: 14.8 s, total: 27min 25s
Wall time: 28min 3s


**Лучший результат на обучающей выборке показала модель LogisticRegression с гиперпараметрами: (class_weight='balanced', max_iter=200, random_state=1) на корпусе обработанным методом CountVectorizer, её и будем тестировать на тестовой выборке**

### Тестирование лучшей модели по результатам обучающей выборке на тестовой

In [29]:
predict = best_model_lin_cv.predict(features_test_cv)
best_test_score = round(f1_score(target_test, predict), 2)
best_test_score

0.75

**Проверка на адекватность модели**

In [30]:
dummy_model = DummyClassifier(strategy='uniform', random_state=RND)
dummy_model.fit(features_train_cv, target_train)
dummy_predict = dummy_model.predict(features_test_cv)
dummy_score = round(f1_score(target_test, dummy_predict), 2)
dummy_score

0.17

Наша модель прошла проверку на адекватность)

## Выводы

In [31]:
rezult = pd.DataFrame(index=['F1 на обучающей выборке', 'F1 на тестовой выборке'],
                      columns=['LogisticRegression_CV', 'LogisticRegression_Tfidf', 'DecisionTreeClassifier_CV',
                      'DecisionTreeClassifier_Tfidf', 'RandomForestClassifier_CV', 'RandomForestClassifier_Tfidf',
                      'CatBoostClassifier_CV', 'CatBoostClassifier_Tfidf', 'DummyClassifier'])
rezult['LogisticRegression_CV'] = best_score_lin_cv, best_test_score
rezult['DecisionTreeClassifier_CV'] = best_score_tree_cv, ''
rezult['RandomForestClassifier_CV'] = best_score_forest_cv, ''
rezult['CatBoostClassifier_CV'] = best_score_cat_cv, ''
rezult['LogisticRegression_Tfidf'] = best_score_lin_tfv, ''
rezult['DecisionTreeClassifier_Tfidf'] = best_score_tree_tfv, ''
rezult['RandomForestClassifier_Tfidf'] = best_score_forest_tfv, ''
rezult['CatBoostClassifier_Tfidf'] = best_score_cat_tfv, ''
rezult['DummyClassifier'] = '', dummy_score
rezult.T

,F1 на обучающей выборке,F1 на тестовой выборке
LogisticRegression_CV,0.75276,0.75
LogisticRegression_Tfidf,0.743189,
DecisionTreeClassifier_CV,0.655915,
DecisionTreeClassifier_Tfidf,0.623163,
RandomForestClassifier_CV,0.363964,
RandomForestClassifier_Tfidf,0.36787,
CatBoostClassifier_CV,0.685554,
CatBoostClassifier_Tfidf,0.682882,
DummyClassifier,,0.17


* **На этапе знакомства с данными и их предобработкой были подведены такие итоги:**
    * данные импортированы с удалением лишнего индекса при чтении
    * дубликатов, пропусков нету
    * в данных виден большой дисбаланс классов
    * проведена лемматизация данных и удаление лишних символов
    * данные поделены на две выборки: обучающую и тестовую
    * удалены стоп-слова из признаков
    * получены мешки слов в признаках двумя способами
* **На этапе обучения моделей:**
    * обучены 4 модели на двух разных мешках признаков, полученными методами CountVectorizer и TfidVectorizer
    * лучший результат на обучающей выборке показала модель LogisticRegression с гиперпараметрами: class_weight='balanced', max_iter=200, random_state=1 на признаках полученными методом CountVectorizer
    * на тестовой выборке лучшая модель показала метрику F1 = 0.75
    * проверили модель на адекватность, модель DummyClassifier показала метрику F1 = 0.17 - это означает наша модель работает правильно, так как если сопоставить точность предсказаний модели LogisticRegression с точностью предсказаний модели DummyClassifier, то результат нашей подобранной модели значительно выше результата Dummy модели
* **Для выполнения поставленной задачи интернет-магазина «Викишоп» подходит модель *LogisticRegression.*** 